In [154]:
from tkinter import *
import math
import datetime
import numpy as np
import time
import pandas as pd
def getSEA(latitude, longitude, utc_offset):
    date = datetime.datetime.now().timetuple()
    hour = 12
    minute = 0
    # Check your timezone to add the offset
    hour_minute = (hour + minute / 60) - utc_offset
    day_of_year = date[7]

    g = (360 / 365.25) * (day_of_year + hour_minute / 24)

    g_radians = math.radians(g)

    declination = 0.396372 - 22.91327 * math.cos(g_radians) + 4.02543 * math.sin(g_radians) - 0.387205 * math.cos(
        2 * g_radians) + 0.051967 * math.sin(2 * g_radians) - 0.154527 * math.cos(3 * g_radians) + 0.084798 * math.sin(
        3 * g_radians)

    time_correction = 0.004297 + 0.107029 * math.cos(g_radians) - 1.837877 * math.sin(g_radians) - 0.837378 * math.cos(
        2 * g_radians) - 2.340475 * math.sin(2 * g_radians)

    SHA = (hour_minute - 12) * 15 + longitude + time_correction

    if (SHA > 180):
        SHA_corrected = SHA - 360
    elif (SHA < -180):
        SHA_corrected = SHA + 360
    else:
        SHA_corrected = SHA

    lat_radians = math.radians(latitude)
    d_radians = math.radians(declination)
    SHA_radians = math.radians(SHA)

    SZA_radians = math.acos(
        math.sin(lat_radians) * math.sin(d_radians) + math.cos(lat_radians) * math.cos(d_radians) * math.cos(
            SHA_radians))

    SZA = math.degrees(SZA_radians)

    SEA = 90 - SZA
    return SEA


In [167]:
class Sun:
    def __init__(self, SEA):
        self.r = 1000
        self.x = self.r * math.cos(SEA)
        self.y = self.r * math.sin(SEA)
    def position(self):
        self.x = self.r * math.cos(SEA)
        self.y = self.r * math.sin(SEA)
        position = [self.x,self.y]
        return np.array(position) 
    def is_rise(self):
        if(self.z > 0):
            return True
        else:
            return False
class Flower:
    def __init__(self):
        self.x = 0
        self.y = 0
    def position(self):
        return np.array([self.x, self.y])
class Cloude:
    def __init__(self):
        self.x1 = 500
        self.x2 = 650
        self.y1 = 300
        self.y2 = 300
        self.speed_of_wind_x  = 20
        self.speed_of_wind_y = 0
    def position(self):
        return np.array([self.x1, self.y1, self.x2, self.y2])
    def move(self):
        self.y1 = self.y1 + self.speed_of_wind_y
        self.y2 = self.y2 + self.speed_of_wind_y
        self.x1 = self.x1 + self.speed_of_wind_x
        self.x2 = self.x2 + self.speed_of_wind_x
class Lamp:
    def __init__(self):
        self.ligth = False
    def on(self):
        self.ligth = True
        #print("Lamp on")
    def off(self):
        self.ligth = False
        #print("Lamp off")
    def check(self):
        if(self.ligth):
            return "ON"
        else:
            return "OFF"
test_cloude = Cloude()
test_cloude.position()

array([500, 300, 650, 300])

In [168]:
def intersection(sun, cloude, flower): 
        vector_of_sun = sun.position() - flower.position()
        vector_of_cloude = np.array([cloude.position()[2] - cloude.position()[0], cloude.position()[3] - cloude.position()[1]])
        #считаем уравнения прямых проходящих через отрезки
        a1 = -vector_of_sun[1]
        b1 = +vector_of_sun[0]
        d1 = -(a1*sun.x + b1*sun.y)

        a2 = -vector_of_cloude[1]
        b2 = +vector_of_cloude[0]
        d2 = -(a2*cloude.x1 + b2*cloude.y1)

        #подставляем концы отрезков, для выяснения в каких полуплоскотях они
        seg1_line2_start = a2*sun.x + b2*sun.y + d2
        seg1_line2_end = a2*flower.x + b2*flower.y + d2

        seg2_line1_start = a1*cloude.x1 + b1*cloude.y1 + d1
        seg2_line1_end = a1*cloude.x2 + b1*cloude.y2 + d1

        #если концы одного отрезка имеют один знак, значит он в одной полуплоскости и пересечения нет.
        if (seg1_line2_start * seg1_line2_end >= 0 or seg2_line1_start * seg2_line1_end >= 0):
            return False
        #u = seg1_line2_start / (seg1_line2_start - seg1_line2_end)
        #out_intersection =  + u*vector_of_sun

        return True
if(intersection(sun,cloude,flower)):
    print("Пересечение")
else:
    print("нет пересечений")

Пересечение


In [169]:
#Field
SEA = math.radians(getSEA(55.7522, 37.6156, 3))
sun = Sun(SEA)
cloude = Cloude()
flower = Flower()
lamp = Lamp()
time_on = list()
status = list()
print(sun.position())
print(cloude.position())
def update_field(sun, flower, cloude, lamp):
    cloude.move()
    sun.position()
for t in range(10):
    time_on.append(t)
    status.append(lamp.check())
    update_field(sun, flower, cloude,lamp)
    if(intersection(sun,cloude,flower)):
        lamp.on()
    else:
        lamp.off()
    time.sleep(1)
df = pd.DataFrame({'Время с начала запуска': time_on, 'Состояние системы': status})
df.to_excel('Отчет о работе ламы.xlsx')
df.head()

[930.48722706 366.32433755]
[500 300 650 300]


,Время с начала запуска,Состояние системы
0,0,OFF
1,1,OFF
2,2,OFF
3,3,OFF
4,4,OFF


In [165]:

    def drowing(self):
        root = Tk()
        canv = Canvas(root, width = 1000, height = 1000, bg = "lightblue", cursor = "pencil")
        canv.create_line(500,1000,500,0,width=2,arrow=LAST) 
        canv.create_line(0,500,1000,500,width=2,arrow=LAST) 
        canv.create_line(-538,  -842, self.flower.x+500, self.flower.y+500,  width = 2, arrow = LAST)
        canv.pack()
        root.mainloop()